This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is Dilated RNN as appears in https://arxiv.org/abs/1710.02224
and using the code available on https://github.com/zalandoresearch/pt-dilate-rnn

# todos:

* save/load model
* load pre-trained embedding


In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score
import plotly.plotly as py 
import plotly.graph_objs as go
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace

# for windows only    
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


In [2]:
use_cuda 

True

In [3]:
# prepare data: load the input file containing list of list of list, and corresponding label file
# and output the splitted training, valid and Test sets

def data_load_split_VT(seqFile = 'data/cerner/hospital_data/h143.visits', labelFile = 'data/cerner/hospital_data/h143.labels' , test_r=0.2 , valid_r=0.1):

    set_x = pickle.load(open(seqFile, 'rb'), encoding='bytes')
    set_y = pickle.load(open(labelFile, 'rb'),encoding='bytes')
    merged_set = [[set_y[i],set_x[i]] for i in range(len(set_x))] # merge the two lists

    # set random seed
    random.seed( 3 )
    
    dataSize = len(merged_set)
    nTest = int(test_r * dataSize)
    nValid = int(valid_r * dataSize)
    
    random.shuffle(merged_set)

    test_set = merged_set[:nTest]
    valid_set = merged_set[nTest:nTest+nValid]
    train_set = merged_set[nTest+nValid:]

    return train_set, valid_set, test_set

In [4]:
train_sl , valid_sl , test_sl = data_load_split_VT()

#print (train_sl[0:5])


In [5]:
class DRNN(nn.Module):

    def __init__(self, n_input, n_hidden, n_layers, dropout=0, cell_type='GRU'):

        super(DRNN, self).__init__()

        self.dilations = [2 ** i for i in range(n_layers)]
        self.cell_type = cell_type
        self.D = n_input
        self.embedBag = nn.EmbeddingBag(20000,self.D ,mode= 'sum')

        self.cells = nn.ModuleList([])

        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError

        for i in range(n_layers):
            if i == 0:
                c = cell(n_input, n_hidden, dropout=dropout)
            else:
                c = cell(n_hidden, n_hidden, dropout=dropout)
            self.cells.append(c)

        self.out = nn.Linear(n_hidden,1)

    

    def EmbedPatient_MB(self, seq_mini_batch): # x is a ehr_seq_tensor
        
       
        lp= len(max(seq_mini_batch, key=lambda xmb: len(xmb[1]))[1])
        tb= torch.FloatTensor(len(seq_mini_batch),lp,self.D) 
        lbt1= torch.FloatTensor(len(seq_mini_batch),1)

        for pt in range(len(seq_mini_batch)):
              
            lbt ,pt_visits =seq_mini_batch[pt]
            lbt1[pt] = torch.FloatTensor([[float(lbt)]])
            ml=(len(max(pt_visits, key=len))) ## getting the max number of visits for pts within the minibatch
            txs= torch.LongTensor(len(pt_visits),ml)
            
            b=0
            for i in pt_visits:
                pd=(0, ml-len(i))
                txs[b] = F.pad(torch.from_numpy(np.asarray(i)).view(1,-1),pd,"constant", 0).data
                b=b+1
            if use_cuda:
                txs=txs.cuda()
                
            emb_bp= self.embedBag(Variable(txs)) ### embed will be num_of_visits*max_num_codes*embed_dim 
            #### the embed Bag dim will be num_of_visits*embed_dim
            
            #print ('embed bag Matrix : /n' , emb_bp )
            
            zp= nn.ZeroPad2d((0,0,0,(lp-len(pt_visits))))
            xzp= zp(emb_bp)

            #print ('padded embed bag Matrix : /n' , xzp )

            tb[pt]=xzp.data
        
        #print ('pts embed bag Matrix : /n' , tb )
        
        # input for RNN need to be seq_len, batch, input_size
        tb= tb.permute(1, 0, 2)  ### as my final input need to be seq_len x batch_size x input_size 
        
        #print (tb.size())

        #print ('Final input : /n' , tb )
        
        emb_m=Variable(tb)
        label_tensor = Variable(lbt1)

        if use_cuda:
                label_tensor = label_tensor.cuda()
                emb_m = emb_m.cuda()
        
        #print ('just for verificaton: /n Label tensor var: /n', label_tensor , 'input emb : /n', emb_m , 'input reformat done')
        return emb_m , label_tensor

    def forward(self, inputs, hidden=None):
        
        x , lt = self.EmbedPatient_MB(inputs)
        outputs = []
        for i, (cell, dilation) in enumerate(zip(self.cells, self.dilations)):
            if hidden is None:
                x = self.drnn_layer(cell, x, dilation)
            else:
                x = self.drnn_layer(cell, x, dilation, hidden[i])
            
            outputs.append(x[-dilation:])

        #x= F.sigmoid(F.max_pool1d(self.out(x)))
        #x = self.out(x).squeeze()
        #print ('x dim', x.size())
        x = F.sigmoid(F.max_pool1d(self.out(x).permute(2,1,0),x.size(0)))

        return x.squeeze(), lt.squeeze()

    def drnn_layer(self, cell, inputs, rate, hidden=None):

        #n_steps = len(inputs)
        n_steps = inputs.size(0)
        #print('n_steps',n_steps) 
        #batch_size = inputs[0].size(0)
        batch_size = inputs.size(1)
        #print('batch size',batch_size) --verified correct
        hidden_size = cell.hidden_size
        #print('hidden size',hidden_size) --verified correct

        inputs, dilated_steps = self._pad_inputs(inputs, n_steps, rate)
        dilated_inputs = self._prepare_inputs(inputs, rate)

        if hidden is None:
            dilated_outputs = self._apply_cell(dilated_inputs, cell, batch_size, rate, hidden_size)
        else:
            hidden = self._prepare_inputs(hidden, rate)
            dilated_outputs = self._apply_cell(dilated_inputs, cell, batch_size, rate, hidden_size, hidden=hidden)

        splitted_outputs = self._split_outputs(dilated_outputs, rate)
        outputs = self._unpad_outputs(splitted_outputs, n_steps)

        return outputs
    
       
    def _apply_cell(self, dilated_inputs, cell, batch_size, rate, hidden_size, hidden=None):

        if hidden is None:
            if self.cell_type == 'LSTM':
                c, m = self.init_hidden(batch_size * rate, hidden_size)
                hidden = (c.unsqueeze(0), m.unsqueeze(0))
            else:
                hidden = self.init_hidden(batch_size * rate, hidden_size).unsqueeze(0)

        dilated_outputs = cell(dilated_inputs, hidden)[0]

        return dilated_outputs

    def _unpad_outputs(self, splitted_outputs, n_steps):

        return splitted_outputs[:n_steps]

    def _split_outputs(self, dilated_outputs, rate):

        batchsize = dilated_outputs.size(1) // rate

        blocks = [dilated_outputs[:, i * batchsize: (i + 1) * batchsize, :] for i in range(rate)]

        interleaved = torch.stack((blocks)).transpose(1, 0).contiguous()
        interleaved = interleaved.view(dilated_outputs.size(0) * rate,
                                       batchsize,
                                       dilated_outputs.size(2))
        return interleaved

    def _pad_inputs(self, inputs, n_steps, rate):

        iseven = (n_steps % rate) == 0

        if not iseven:
            dilated_steps = n_steps // rate + 1

            zeros_ = torch.zeros(dilated_steps * rate - inputs.size(0),
                                 inputs.size(1),
                                 inputs.size(2))
            if use_cuda:
                zeros_ = zeros_.cuda()

            inputs = torch.cat((inputs, autograd.Variable(zeros_)))
        else:
            dilated_steps = n_steps // rate

        return inputs, dilated_steps

    def _prepare_inputs(self, inputs, rate):

        dilated_inputs = torch.cat([inputs[j::rate, :, :] for j in range(rate)], 1)


        return dilated_inputs

    def init_hidden(self, batch_size, hidden_dim):
        c = autograd.Variable(torch.zeros(batch_size, hidden_dim))
        if use_cuda:
            c = c.cuda()
        if self.cell_type == "LSTM":
            m = autograd.Variable(torch.zeros(batch_size, hidden_dim))
            if use_cuda:
                m = m.cuda()
            return (c, m)
        else:
            return c

In [6]:
model = DRNN(128,64, 5, 0, 'GRU')
if use_cuda:
    model  = model.cuda()


In [7]:
def train (mini_batch, criterion, optimizer):  
    
    model.zero_grad()
    
    output , label_tensor = model (mini_batch)
    

    loss = criterion(output, label_tensor)
    
    loss.backward()
    
    optimizer.step()
    
   
    return output, loss.data[0]


In [8]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)






In [9]:
def run_model_train(dataset,batch_size,learning_rate = 0.001 ):
    
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    #optimizer = optim.Adadelta(model.parameters(), '''lr=learning_rate,''' weight_decay=0)
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=learning_rate)
    optimizer = optim.Adam(model.parameters(), weight_decay=learning_rate)
    #optimizer = optim.RMSprop (model.parameters(),lr=learning_rate,weight_decay=0.9)### same as reported in the paper , really poor
    dataset.sort(key=lambda pt:len(pt[1])) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    #print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Print iter number, loss, name and guess
            #if iter % print_every == 0:
               #print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses

    
    

In [10]:
def calculate_auc(test_model, dataset, batch_size=200):

    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []

    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc


In [ ]:
epochs=20
batch_size=200
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]

valid_auc_allep =[]

test_auc_allep=[]


for ep in range(epochs):
    
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(train_sl,batch_size)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc, " Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
    
    current_loss_l.append(current_loss_la)


In [ ]:
import plotly.plotly as py 
import plotly.graph_objs as go
py.sign_in('LailaRasmy','mzNHzVvwYjcZwBDZx3B7')

train_auc_fg= go.Scatter(x= np.arange(epochs), y=train_auc_allep, name='train')
test_auc_fg= go.Scatter(x= np.arange(epochs), y=test_auc_allep, name='test')
valid_auc_fg= go.Scatter(x= np.arange(epochs), y=valid_auc_allep, name='valid')
valid_max = max(valid_auc_allep)
test_max = max(test_auc_allep)
data = [train_auc_fg,test_auc_fg,valid_auc_fg]#,valid_auc_allep,test_auc_allep] 
layout = go.Layout(xaxis=dict(dtick=1))
layout.update(dict(annotations=[go.Annotation(text="Max Valid", x=valid_auc_allep.index(valid_max), y=valid_max)]))
#layout.update(dict(annotations=[go.Annotation(text="Max Test", x=test_auc_allep.index(test_max), y=test_max)]))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='DRNN_Auc')
#url = py.plot(data, filename='some-data')  # gen. online plot
#py.image.save_as(data, 'some-data.png') 